In [1]:
#Importing dependencies.
from splinter import Browser
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import time
import sqlalchemy
from sqlalchemy import create_engine, Column, Integer, String, PrimaryKeyConstraint, func,inspect
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, Session
import sqlite3
from sqlalchemy.ext.automap import automap_base

#Use Splinter to establish a browser that connects to the pickleheads website.
url = 'https://www.pickleheads.com/courts/us/'
us_city_df = pd.read_csv('uscities.csv')
state_df=pd.read_csv('states.csv')
all_states=[]
for i,state in state_df.iterrows():
    state_url=url+state_df['state'][i]
    all_states.append({'State':state_df['state'][i],'URL':state_url})

# Define the URL template for the state pages
url_template = 'https://www.pickleheads.com/courts/us/{}'
column_data_types = {
    'ID': Integer,
    'Latitude': String,
    'Longitude': String,
    'Location Name': String,
    'City': String,
    'Number of Courts': Integer 
}

# Loop through all the state URLs
for state_data in all_states:
    state_name = state_data['State']
    state_url = state_data['URL']

    # Use Splinter to establish a browser that connects to the state's pickleheads website.
    browser = Browser('chrome')
    browser.visit(state_url)

    # Click on the "Show More" button if it exists
    buttons = browser.find_by_css(".css-2ttlle")
    button_index_to_click = 1
    if button_index_to_click < len(buttons):
        button_to_click = buttons[button_index_to_click]
        button_to_click.click()
        time.sleep(2)

    # Get the HTML content after clicking the button
    html = browser.html

    # Create a Beautiful Soup object from the HTML content and parse the information.
    soup = BeautifulSoup(html, 'html.parser')

    # Find the section of interest that has the class css-12eoos1 and save it as a variable.
    whole_list = soup.find('div', class_='css-12eoos1')

    # Initialize lists to store data
    location = []
    city = []
    courts = []

    # Using a for loop, find the location name, city name, and number of courts and append them to their respective lists.
    for entry in whole_list:
        location_data = entry.find('span', class_='css-uyrttv')
        if location_data:
            location.append(location_data.text)
        else:
            continue

        city_data = entry.find(class_='css-1vl3ez2')

        if city_data:
            city.append(city_data.text)

        court_data = entry.select('.css-1vl3ez2')
        if court_data:
            last_span = court_data[-1]
            courts.append(last_span.text)

    # Create a dataframe with the locations and cities using the three lists
    pickleball_dict = {"Location Name": location, "City": city, "Number of Courts": courts}
    state_pickleball_df = pd.DataFrame(pickleball_dict)

    # Sorted the pickleball dataframe by city.
    state_pickleball_df_sorted = state_pickleball_df.sort_values(by='City', ascending=True)

    # Merge the state pickleball dataframe with the city county dataframe
    state_full_info = pd.merge(state_pickleball_df_sorted, us_city_df, on='City', how='left')

    # Remove the word "court" or "courts" from the Number of Courts column
    state_full_info['Number of Courts'] = state_full_info['Number of Courts'].str.replace(' courts', '')
    state_full_info['Number of Courts'] = state_full_info['Number of Courts'].str.replace(' court', '')

    # Add a column called 'ID' to be used as a primary key
    state_full_info['ID'] = range(1, len(state_full_info) + 1)

    #Temporarily using a csv to check if all the cities are in the US cities csv
    state_full_info.to_csv('state_full_info.csv', index=False)

    # # Convert the state pickleball dataframe to a SQLite table
    # engine = create_engine('sqlite:///state_courts_db.sqlite')
    # table_name = state_name.lower() + '_courts'
    # state_full_info.to_sql(table_name, engine, index=False, if_exists='replace', dtype=column_data_types)

    # Close the browser
    browser.quit()
